In [1]:
import os
import json
import pandas as pd

In [22]:
# create empty lists to store rows for table 1, 2, 3
table1_rows = []
table2_rows = []
table3_rows = []
table4_rows = []
        
# Lists for generating table 1    
fileName=[]
folderName=[]
width=[]
height=[]
    
# Loop through all JSON files in current directory
for filename in os.listdir(os.getcwd()):
    if filename.endswith('.json'):
        with open(filename, 'r') as f:
            data = json.load(f)

            # extract folder name, filename, orientation for Table 1
            for photo in data:
                folder_name = photo['image']['details']['folderName']
                file_name = photo['image']['fileName']
                width = photo['image']['details']['width']
                height = photo['image']['details']['height']
                orientation = 'Landscape' if width > height else 'Portrait'    
                table1_rows.append([folder_name, file_name, orientation])
            # iterate through each photo in the data to extract information about the monkeys and toys in each photo
            for photo in data:
                folder_name = photo['image']['details']['folderName']
                file_name = photo['image']['fileName']

                # extract monkey / toy
                for obj in photo['objects']:
                    category_name = obj['category']['name']

                    # calculate the coordinates of the bounding box for the object
                    coordinates = [(int(obj['bndbox']['minX'] * width), int(obj['bndbox']['minY'] * height)),
                                   (int(obj['bndbox']['maxX'] * width), int(obj['bndbox']['minY'] * height)),
                                   (int(obj['bndbox']['maxX'] * width), int(obj['bndbox']['maxY'] * height)),
                                   (int(obj['bndbox']['minX'] * width), int(obj['bndbox']['maxY'] * height))]

                    # if object is monkey, extract monkeyID and Face/Body
                    if 'Face' in category_name or 'Body' in category_name:
                        monkey_id = category_name.split()[0]
                        face_or_body = category_name.split()[1]
                        table2_rows.append([file_name, monkey_id, face_or_body, coordinates])

                    # if the object is a toy, record its name and coordinates in table 3
                    elif 'Toy' in category_name:
                        toy_name = category_name[4:]
                        table3_rows.append([file_name, toy_name, coordinates])
                        
                    else:
                        table4_rows.append(file_name, "Error")
        if not data:
            print("Error in retrieving data in", file_name)

In [23]:
# create dataframes from the rows and columns for table 2 and table 3
table1 = pd.DataFrame.from_records(table1_rows, columns=['Folder Name', 'Image File Name', 'Orientation'])
table2 = pd.DataFrame.from_records(table2_rows, columns=['Image File Name', 'Monkey ID', 'Face/Body', 'Coordinates'])
table3 = pd.DataFrame.from_records(table3_rows, columns=['Image File Name', 'Toy Name', 'Coordinates'])

In [24]:

print(table1)
print(table2)
print(table3)

    Folder Name         Image File Name Orientation
0        0415ST  20220415_SD01_4120.JPG   Landscape
1        0415ST  20220415_SD01_4112.JPG    Portrait
2        0415ST  20220415_SD01_4110.JPG    Portrait
3        0415ST  20220415_SD01_4119.JPG    Portrait
4        0415ST  20220415_SD01_4109.JPG   Landscape
..          ...                     ...         ...
209      0512IG  20220512_SD02_6001.JPG    Portrait
210      0512IG  20220512_SD02_6025.JPG    Portrait
211      0512IG  20220512_SD02_6061.JPG    Portrait
212      0512IG  20220512_SD02_5971.JPG    Portrait
213      0512IG  20220512_SD02_6106.JPG    Portrait

[214 rows x 3 columns]
            Image File Name Monkey ID Face/Body  \
0    20220415_SD01_4120.JPG       58I      Face   
1    20220415_SD01_4120.JPG       58I      Body   
2    20220415_SD01_4112.JPG       58I      Face   
3    20220415_SD01_4112.JPG       58I      Body   
4    20220415_SD01_4110.JPG       58I      Body   
..                      ...       ...       ..